# 3.Marketing_Case-ELVT-Deploy

# Scikit-LEARN

In this 3rd and last notebook we will deploy our model into Azure Machine learning service

In [19]:
import sys
import pickle
import os
import json
import pandas as pd
#!{sys.executable} -m pip install azureml-sdk --user

In [29]:
MODEL_NAME = 'ELVTmodelsklearn'

FILE = 'marketing_training_cleaned.csv'
TEST_DF_FILE = './marketing_test_sklearn.pkl'
MODEL_FILE = './marketing_model.pkl'
NORMALIZATION_DICT = './normalization_dict.pkl' 

MODEL_PATH = MODEL_FILE #taking first model from folder

## Load Test data

In [31]:
with open(TEST_DF_FILE, 'rb') as handle:
    test = pickle.load(handle)
test

,pcontacted_pdays,cons.conf.idx_scaled,cons.price.idx_scaled,contact_telephone,day_of_week_thu,day_of_week_unknown,day_of_week_fri,day_of_week_mon,day_of_week_wed,day_of_week_tue,...,campaign1_2,campaign1_7+,campaign1_4,campaign1_1,campaign1_5,campaign1_3,custAge1_scaled,pastEmail1_no,pContacted_previous_no,response
5602,1,0.376569,0.882307,1,0,1,0,0,0,0,...,0,1,0,0,0,0,0.586957,0,0,0
1533,1,0.368201,0.389322,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0.500000,0,0,0
182,1,0.602510,0.698753,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0.326087,0,0,0
3862,1,0.154812,0.340608,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0.478261,1,1,0
611,1,0.192469,0.269680,1,0,1,0,0,0,0,...,0,0,0,1,0,0,0.239130,0,0,0


## Load model  

In [22]:
with open(MODEL_FILE, 'rb') as handle:
    model = pickle.load(handle)
model

GaussianNB(priors=None)

In [34]:
with open(NORMALIZATION_DICT, 'rb') as handle:
    normalization_dict = pickle.load(handle)
print('Loaded')
normalization_dict

Loaded


{'campaign1': LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
 'cons.conf.idx': MinMaxScaler(copy=True, feature_range=(0, 1)),
 'cons.price.idx': MinMaxScaler(copy=True, feature_range=(0, 1)),
 'contact': LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
 'custAge1': MinMaxScaler(copy=True, feature_range=(0, 1)),
 'day_of_week': LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
 'default': LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
 'emp.var.rate': MinMaxScaler(copy=True, feature_range=(0, 1)),
 'euribor3m': MinMaxScaler(copy=True, feature_range=(0, 1)),
 'housing': LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
 'loan': LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
 'marital': LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
 'month': LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
 'nr.employed': MinMaxScaler(copy=True, feature_range=(0, 1)),
 'pContacted_previou

## Create files for AZML service

In order to make the doployment we need to create a docker image that contains all the logic and environment to run the model. To create the image with h2o it is necesary to provide 2 files 

    1) "score.py", the execution_script to make requests to the model 
    2) "myenv.yml", configuration file for conda environment.

### env.yml

In [24]:
%%writefile myenv.yml
name: myenv
channels:
  - defaults
dependencies:
  - python=3.6
  - pip:
    # Required packages for AzureML execution, history, and data preparation.
    - sklearn
    - azureml-sdk
    - azureml-core

Overwriting myenv.yml


### score.py

score.py contains two main functions:
    
    - init(), it will be called the first time the container is launched 
    - run(), it is called every time a request is made

In [36]:
%%writefile score.py

import pickle
import json
import numpy as np
import pandas as pd
from azureml.core.model import Model
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler

MODEL = 'model'

def init():
    # read in the model file
    global model
    global normalization_dict

    model_path = Model.get_model_path(model_name = MODEL)
    
    # load model
    with open(MODEL_FILE, 'rb') as handle:
        model = pickle.load(handle)
    print("Model Loaded")
    
    # Load normalization values
    with open(NORMALIZATION_DICT, 'rb') as handle:
        normalization_dict = pickle.load(handle)
    print('Loaded')
        
def run(raw_data):
    try:
        data = json.loads(raw_data)['data']
        data = pd.read_json(data, orient='records')
        
        for column in normalization_dict:
            normalization_dict[column].fit_transform(data[column].values)
            name = data[column].name
            dfOneHot = pd.DataFrame(transformed, 
                                    columns =[name+"_"+str(list(set(data[column].values))[i]) for i in range(transformed.shape[1])])
            data = pd.concat([data, dfOneHot], axis=1)

        pred = model.predict(data)
        print(pred)
        
        # Send results
        pred = pred.as_data_frame().values.tolist()
        return json.dumps({"result": pred})

    except Exception as e:
        result = str(e)
        return json.dumps({"error": result})

Overwriting score.py


## Register model on Azure

In [37]:
import azureml.core
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core.webservice import AciWebservice

print("SDK Version:", azureml.core.VERSION)

SDK Version: 1.0.2


In [38]:
# subscription_id = "<SUBSCRIPTION-ID>"
# resource_group = "aml_rg"
# workspace_name = "aml_workspace"
# workspace_region = 'southcentralus'

# #Only one time: create the AML Workspace
# ws = Workspace.create(name = workspace_name,
#                       subscription_id = subscription_id,
#                       resource_group = resource_group, 
#                       location = workspace_region)
# ws.get_details()
# ws.write_config()

# load workspace configuratio from ./aml_config/config.json file
ws = Workspace.from_config()
print('Name: '+ws.name, 'Resource Group: '+ws.resource_group, 'Location: '+ws.location, sep = '\n')

Found the config file in: C:\JP_NOTEBOOKS\aml_config\config.json
Name: aml_workspace
Resource Group: aml_rg
Location: westcentralus


In [39]:
model = Model.register(model_path = MODEL_PATH,
                       model_name = MODEL_NAME,
                       tags = {'framework': 'sklearn', 'Department':'Marketing' ,'target': "Responded"},
                       description = "Classification model using sklearn for marketing purposes",
                       workspace = ws)
print('Model Registered')



Registering model ELVTmodelsklearn
Model Registered


In [41]:
model = Model.register(model_path = TEST_DF_FILE,
                       model_name = 'marketing_test_sklearn',
                       tags = {'framework': 'sklearn', 'Department':'Marketing' ,'target': "Responded"},
                       description = "Test DF using sklearn for marketing purposes",
                       workspace = ws)
print('TESTDF Registered')

model = Model.register(model_path = NORMALIZATION_DICT,
                       model_name = 'normalization_dict_sklearn',
                       tags = {'framework': 'sklearn', 'Department':'Marketing' ,'target': "Responded"},
                       description = "normalization_dict sklearn for marketing purposes",
                       workspace = ws)
print('NORMALIZATION DICT Registered')



Registering model marketing_test_sklearn
TESTDF Registered
Registering model normalization_dict_sklearn
NORMALIZATION DICT Registered


In [42]:
for key,value in ws.models.items():
    print("Name:", key,"\tVersion:", value.version)

Name: normalization_dict_sklearn 	Version: 1
Name: ELVTmodel 	Version: 3
Name: ELVTmodelsklearn 	Version: 2
Name: marketing_test_sklearn 	Version: 1
Name: model 	Version: 1


## Create Docker image and store it on azure

In [45]:
%%time
from azureml.core.image import ContainerImage

image_config = ContainerImage.image_configuration(execution_script = "score.py",
                                                    runtime = "python",
                                                    conda_file = "myenv.yml",
                                                    tags = {'framework': 'sklearn', 'Department':'Marketing' ,'target': "Responded"},
                                                    description = "Classification model using sklearn for marketing purposes")

image = ContainerImage.create(name = "elvtsklearn.image",
                              # this is the model object
                              models = [model],
                              image_config = image_config,
                              workspace = ws)

image.wait_for_creation(show_output = True)

Creating image
Running..................
SucceededImage creation operation finished for image elvtsklearn.image:2, operation "Succeeded"
Wall time: 1min 43s


## Deploy as web service on Azure Container Instance 

In [47]:
aci_service_name = 'aci-elvt'

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 4, 
                                               tags = {'framework': 'sklearn', 'Department':'Marketing' ,'target': "Responded"},
                                               description = "Classification model using h2o for marketing purposes")


In [ ]:
%%time
# create an Azure Container Instance service 

aci_service = AciWebservice.deploy_from_image(deployment_config = aciconfig,
                                       image = image,
                                       name = aci_service_name,
                                       workspace = ws)
aci_service.wait_for_deployment(True)
print(aci_service.state)


In [ ]:
#print(aci_service.get_logs())

In [ ]:
print(aci_service.name)
print(aci_service.state)
print(aci_service.location)
print(aci_service.image_id)
print(aci_service.scoring_uri)
print(aci_service.description)
print(aci_service.tags)

## Testing

In [ ]:
print('web service hosted in ACI:', aci_service.scoring_uri)

In [49]:
test_json = json.dumps({"data": test.to_json(orient='records')})
display(test_json)

'{"data": "[{\\"pcontacted_pdays\\":1,\\"cons.conf.idx_scaled\\":0.3765690377,\\"cons.price.idx_scaled\\":0.8823070928,\\"contact_telephone\\":1,\\"day_of_week_thu\\":0,\\"day_of_week_unknown\\":1,\\"day_of_week_fri\\":0,\\"day_of_week_mon\\":0,\\"day_of_week_wed\\":0,\\"day_of_week_tue\\":0,\\"default_unknown\\":0,\\"emp.var.rate_scaled\\":1.0,\\"euribor3m_scaled\\":0.9591929268,\\"housing_unknown\\":1,\\"housing_no\\":0,\\"housing_yes\\":0,\\"loan_unknown\\":1,\\"loan_no\\":0,\\"loan_yes\\":0,\\"marital_unknown\\":0,\\"marital_married\\":1,\\"marital_single\\":0,\\"marital_divorced\\":0,\\"month_sep\\":0,\\"month_mar\\":0,\\"month_aug\\":0,\\"month_may\\":0,\\"month_nov\\":1,\\"month_dec\\":0,\\"month_jun\\":0,\\"month_oct\\":0,\\"month_jul\\":0,\\"month_apr\\":0,\\"nr.employed_scaled\\":1.0,\\"poutcome_success\\":0,\\"poutcome_nonexistent\\":1,\\"poutcome_failure\\":0,\\"profession_housemaid\\":0,\\"profession_unknown\\":0,\\"profession_student\\":0,\\"profession_services\\":0,\\"pr

In [ ]:
prediction = aci_service.run(input_data = test_json)
print(prediction)